input- learning rate, the number of neurons in the input, and the output layer. Number of hidden layers will be one of the following (taken as user input). The initial weights would be assigned with random numbers.

## assuming all hidden layer sizes to be same

In [ ]:
import numpy as np

class NN():
    def __init__(self, input_layer_size = 2, hidden_layer_size = 2, output_layer_size = 2, hidden_layers = 1, lr = 0.1, epoches = 1000):
        self.input_layer_size = input_layer_size
        self.hidden_layer_size = hidden_layer_size
        self.output_layer_size = output_layer_size
        self.hidden_layers = hidden_layers
        self.lr = lr
        self.number_of_epoches = epoches

        # initialize the weights by random numbers
        self.W = [np.random.randn(self.input_layer_size, self.hidden_layer_size)]
        for layer in range(hidden_layers-1):
            self.W.append(np.random.randn(self.hidden_layer_size, self.hidden_layer_size))
        self.W.append(np.random.randn(self.hidden_layer_size, self.output_layer_size))

    def activation(self, x): # sigmoid function
        return (1/(1+np.exp(-x)))
    
    def forward(self, X): # forward propogation
        # X- input
        input_list = X
        for step in range(self.hidden_layers + 1):
            output_list = np.dot(input_list, self.W[step])
            output_list = [self.activation(neuron) for neuron in output_list]
            input_list = output_list
        return output_list
    
    def backward(self, X, y): # backward propogation
        # X- input, y- output
        output_list = self.forward(X)
        error = np.sum((y - output_list)**2/2)
        # W2 = W2 - alpha * (H * ((y-OP) * (OP * (OP - 1))))

        pass